<a href="https://colab.research.google.com/github/rdj0im/rl-atsc/blob/main/main_proj_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install SUMO

In [ ]:
!sudo add-apt-repository -y ppa:sumo/stable
!sudo apt-get update
!sudo apt-get -y install sumo sumo-tools sumo-doc

Check Installation path (Unnecessary)

In [ ]:
!which sumo

Set sumo env vars

In [3]:
import os

# Set environment variable
os.environ['SUMO_HOME'] = '/usr/share/sumo'
os.environ['LIBSUMO_AS_TRACI'] = '1' #Optional: for a huge performance boost (~8x) with Libsumo (No GUI)

Setup working dir repo

In [ ]:
!git clone https://github.com/rdj0im/rl-atsc.git
!mv rl-atsc/* ./
!rm -rf rl-atsc


**SUMO RL**

Install sumo-rl

In [ ]:
!pip install sumo-rl

Main



In [ ]:
import sumo_rl
env = sumo_rl.parallel_env(net_file=r'sumo_files/4x4.net.xml',
                  route_file=r'sumo_files/4x4c1c2c1c2.rou.xml',
                  use_gui=False,
                  num_seconds=60000)
observations = env.reset()
print (env.agents)
# while env.agents:
#     actions = {agent: env.action_space(agent).sample() for agent in env.agents} 
    # print (actions) # this is where you would insert your policy
    # observations, rewards, terminations, truncations, infos = env.step(actions)